<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2026-1/Bonos_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 1193, done.
remote: Counting objects: 100% (425/425), done.
remote: Compressing objects: 100% (247/247), done.
remote: Total 1193 (delta 248), reused 278 (delta 158), pack-reused 768 (from 1)
Receiving objects: 100% (1193/1193), 64.19 MiB | 11.19 MiB/s, done.
Resolving deltas: 100% (753/753), done.
Updating files: 100% (450/450), done.


#Funciones auxiliares

In [2]:
#celda 1
import numpy as np

def talamb(nodos, curva, plazos):
    n = max(plazos.shape[1], 1)
    m = max(nodos.shape[1], 1)
    TC = np.zeros((1, n))
    TL = np.zeros((1, n))
    TF = np.zeros((1, n))

    for j in range(n):
        i = 0
        while i < m - 1:
            if nodos[0, i] <= plazos[0, j] <= nodos[0, i + 1]:
                TC[0, j] = curva[0, i]
                TL[0, j] = curva[0, i + 1]
                TF[0, j] = ((((1 + TL[0, j] * nodos[0, i + 1] / 360) / (1 + TC[0, j] * nodos[0, i] / 360)) **
                            ((plazos[0, j] - nodos[0, i]) / (nodos[0, i + 1] - nodos[0, i])) *
                            (1 + TC[0, j] * nodos[0, i] / 360)) - 1) * 360 / plazos[0, j]
                break
            elif plazos[0, j] < nodos[0, 0]:
                TC[0, j] = curva[0, 0]
                TL[0, j] = curva[0, 0]
                TF[0, j] = curva[0, 0]
                break
            elif plazos[0, j] > nodos[0, m - 1]:
                TC[0, j] = curva[0, m - 1]
                TL[0, j] = curva[0, m - 1]
                TF[0, j] = curva[0, m - 1]
                break
            else:
                i += 1

    return TF


# Definición de parámetros para valorar:
Bonos sin cupón
Bonos M
Bondes D

## Definición parámetros Python

In [3]:
import numpy as np
from datetime import datetime, timedelta

fval = datetime.strptime("20230310", "%Y%m%d") #Fecha de valoración
# Tipo de interpolación (0 para interpolación lineal, 1 para tasa alambrada)
itpl = 0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
#setwd(direc)


#CETES
base = "RiesgosFinancieros/2024-1/Tarea/tasa_guber.txt"


#BONOS M
btasadesc_bm = "RiesgosFinancieros/2024-1/Insumos/tasa_guber.txt"
# Tasas fijas del cupón
tfcupon_bm = np.array([0.065, 0.0675, 0.07, 0.075, 0.078])
# Vencimiento del bono
plazos_bm = np.array([378, 405, 550, 1200, 1800])
# Plazos fijos de cada cupón
plazocupon_bm = np.array([182, 182, 182, 182, 182])
# Posición invertida
contratos_bm = np.array([22000, -29000, 29000, -46000, 10000])
# Nominal
nominal_bm = 100




# BONDES D
btasadescst="RiesgosFinancieros/2024-1/Tarea/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2024-1/Tarea/tfondeo.txt"
plazos_bdm=np.array([358, 405, 550, 1200, 1800])
plazocupon_bdm=np.array([28, 28, 28, 28, 28])
contratos_bdm=np.array([220, -290, 290, -460, 100])
nominal_bdm=100


# Carga de datos

In [6]:
import pandas as pd

#BONOS M

# Carga de datos - supongamos que los datos se encuentran en un archivo CSV llamado "btasadesc.csv"
# Asegúrate de proporcionar la ruta completa al archivo o que el archivo esté en el directorio actual.
# data1 = pd.read_("btasadesc.txt", header=None)
data = pd.read_csv(base, sep="\t", header=None)

# Obtiene el número de filas y columnas de data1
n, m_gov= data.shape

# Crear un DataFrame con los datos sin la primera fila y la primera columna
x_orig = pd.DataFrame(data.values[1:, 1:len(data.values[0])], dtype=float)
x_orig['Date'] = pd.to_datetime(data.values[1:,0], format='%Y%m%d')

# Obtener nodos
nodos_gov = pd.DataFrame(data.values[0, 1:(len(data.values[0]))], dtype=int)


#BONDE D
# Leer datos de tasa_guber.txt
data1 = pd.read_table(base, sep="\t", header=None)
n = data1.shape[0]
m_bd = data1.shape[1]
X_orig = data1.iloc[1:n, 0:m_bd]
X1_orig = pd.DataFrame(data1.iloc[1:n, 1:m_bd], dtype=float)
X1_orig['Date'] = pd.to_datetime(X_orig[0], format='%Y%m%d')
nodos = data1.iloc[0, 1:m_bd]
n -= 1

# Leer datos de tasa_guber_st.txt
data3 = pd.read_table(btasadescst, sep="\t", header=None)
n3 = data3.shape[0]
m3_bd = data3.shape[1]
X3a_orig_bd = data3.iloc[1:n3, 0:m3_bd]
X3_orig_bd = pd.DataFrame(data3.iloc[1:n3, 1:m3_bd], dtype=float)
X3_orig_bd['Date'] = pd.to_datetime(X3a_orig_bd[0], format='%Y%m%d')
nodos3_bd = data3.iloc[0, 1:m3_bd]
n3 -= 1

# # Leer datos de tfondeo.txt
data2 = pd.read_table(btasafondeo, sep="\t", header=None)
n2 = data2.shape[0]
X2_orig = data2.iloc[1:n2, 0:2]
X2_orig_bd = X2_orig.copy()
X2_orig_bd[0] = pd.to_datetime(X2_orig_bd[0], format='%Y%m%d')
X2_orig_bd[1] = X2_orig_bd[1].astype(float)

# # Sucesión de días para tasa fondeo
tfh = pd.date_range(start=min(X2_orig_bd[0]), end=max(X2_orig_bd[0]), freq='D')
tfhd = pd.DataFrame({'fecha': tfh})


# Asegúrate de que ambos DataFrames estén ordenados por la columna de fecha de manera descendente
tfhd = tfhd.sort_values(by='fecha', ascending=True)
X2_orig_bd = X2_orig_bd.sort_values(by=0, ascending=True)  # Asumiendo que la columna 'V1' es ahora la columna 0

# Renombrar columnas para preparar para merge_asof
# tfhd = tfhd.rename(columns={'fecha': 'join_date'})
X2_orig_bd = X2_orig_bd.rename(columns={0: 'fecha'})

# Realizar el rolling join con merge_asof
X2_orig_bd = pd.merge_asof(tfhd, X2_orig_bd, on='fecha', direction='backward')


# Si quieres que el resultado esté en el DataFrame original (tfhd), puedes hacer lo siguiente:
X2_orig_bd = pd.merge(tfhd, X2_orig_bd, on='fecha', how='outer')

X1_orig = X1_orig.rename(columns={0: "fecha"})
# Filtrar tasas de fondeo

tf_act = X2_orig_bd.loc[X2_orig_bd['fecha'] == fval, 1].values[0] / 100
tf_int = X2_orig_bd.loc[(X2_orig_bd['fecha'] <= fval) & (X2_orig_bd['fecha'] >= (fval -  timedelta(int(plazocupon_bdm[0])))), 1] / 100
tf_int=tf_int[::-1]  # se hace este cambio para ordenar de mayor a menor


# Ordenar el DataFrame por la columna de unión
X1_orig = X1_orig.sort_values('Date', ascending=True)

# Agregar la columna join_date al DataFrame X1_orig
X1_orig['join_date'] = X1_orig['Date']

# Ordenar el DataFrame X2a_orig por la columna de unión
X2_orig_bd = X2_orig_bd.sort_values('fecha', ascending=True)

# Hacer el merge (join) de los DataFrames
X2_pr = pd.merge_asof(X1_orig, X2_orig_bd, left_on='Date', right_on='fecha')




# Valuación

# Bonos M

In [42]:
import numpy as np

# Número de cupones a pagar
N = (plazos_bm / plazocupon_bm).astype(int) + 1

# Vector de todos los plazos de todos los contratos
VTplazos = np.zeros((1, int(np.sum(N))))

# Vector de todos los contratos de todos los flujos de todos los contratos
contratosT = np.zeros((1, int(np.sum(N))))

# Vector de todos los nominales de todos los flujos de todos los contratos
nominalT = np.zeros((1, int(np.sum(N))))

# Vector de todos los plazos de cupón de todos los flujos de todos los contratos
plazocuponT = np.zeros((1, int(np.sum(N))))

# Vector de tasas fijas de todos los flujos de todos los contratos
tasafijaT = np.zeros((1, int(np.sum(N))))

# Vector de contratos a final de flujo
ulNomT = np.zeros((1, int(np.sum(N))))

# Vector de plazos iniciales
plazini = plazos_bm - plazocupon_bm * (N - 1)

for j in range(len(plazos_bm)):
  # print(j)
  if j == 0:
      VTplazos[:, 0:int(np.sum(N[0:j+1]))] = np.arange(plazini[j], plazos_bm[j] + plazocupon_bm[j], plazocupon_bm[j])
      contratosT[:, 0:int(np.sum(N[0:j+1]))] = np.arange(contratos_bm[j], contratos_bm[j] + 1)
      plazocuponT[:, 0:int(np.sum(N[0:j+1]))] = plazocupon_bm[j] * np.ones(int(np.sum(N[0:j+1])))
      tasafijaT[:, 0:int(np.sum(N[0:j+1]))] = tfcupon_bm[j] * np.ones(int(np.sum(N[0:j+1])))
      ulNomT[:, N[j]-1] = contratos_bm[j]
  else:
      VTplazos[:, int(np.sum(N[0:j])):int(np.sum(N[0:j+1]))] = np.arange(plazini[j], plazos_bm[j] + plazocupon_bm[j], plazocupon_bm[j])
      contratosT[:, int(np.sum(N[0:j])):int(np.sum(N[0:j+1]))] = np.arange(contratos_bm[j], contratos_bm[j] + 1)
      plazocuponT[:, int(np.sum(N[0:j])):int(np.sum(N[0:j+1]))] = plazocupon_bm[j] * np.ones(int(np.sum(N[j:j+1])))
      tasafijaT[:, int(np.sum(N[0:j])):int(np.sum(N[0:j+1]))] = tfcupon_bm[j] * np.ones(int(np.sum(N[j:j+1])))
      ulNomT[:, int(np.sum(N[0:j+1]))-1] = contratos_bm[j]

Xvp = np.zeros((n, VTplazos.shape[1]))

for i in range(n):
    if itpl == 0:
        # Utiliza interpolación lineal
        Xvp[i, :] = np.interp(VTplazos[0, :],np.array(nodos_gov[0]), np.array(x_orig.drop('Date', axis=1).iloc[0]))
    else:
        # Utiliza la función talamb
        # (Asegúrate de que tengas definida la función talamb)
        Xvp[i, :] = talamb(nodos, x_orig[i, :], VTplazos[0, :])

print(VTplazos)
print(ulNomT)
print(contratosT)
print(tasafijaT)
# print(np.array(x_orig[i][0]))
# print(N)
print(N[0])
ulNomT[:, N[0]]
contratos_bm[0]

[[  14.  196.  378.   41.  223.  405.    4.  186.  368.  550.  108.  290.
   472.  654.  836. 1018. 1200.  162.  344.  526.  708.  890. 1072. 1254.
  1436. 1618. 1800.]]
[[     0.      0.  22000.      0.      0. -29000.      0.      0.      0.
   29000.      0.      0.      0.      0.      0.      0. -46000.      0.
       0.      0.      0.      0.      0.      0.      0.      0.  10000.]]
[[ 22000.  22000.  22000. -29000. -29000. -29000.  29000.  29000.  29000.
   29000. -46000. -46000. -46000. -46000. -46000. -46000. -46000.  10000.
   10000.  10000.  10000.  10000.  10000.  10000.  10000.  10000.  10000.]]
[[0.065  0.065  0.065  0.0675 0.0675 0.0675 0.07   0.07   0.07   0.07
  0.075  0.075  0.075  0.075  0.075  0.075  0.075  0.078  0.078  0.078
  0.078  0.078  0.078  0.078  0.078  0.078  0.078 ]]
3


np.int64(22000)

In [41]:
np.array(x_orig.drop('Date', axis=1).iloc[0])

array([0.07968109, 0.07870234, 0.07769828, 0.07959039, 0.0805102 ,
       0.08105581, 0.08126982, 0.08174737, 0.08541634, 0.08868915,
       0.09203079, 0.09487062, 0.09794559, 0.10221934, 0.10665376,
       0.11132426, 0.13989312, 0.17890244, 0.23055418, 0.29968061])

In [19]:
VTplazos

array([[  14.,  196.,  378.,   41.,  223.,  405.,    4.,  186.,  368.,
         550.,  108.,  290.,  472.,  654.,  836., 1018., 1200.,  162.,
         344.,  526.,  708.,  890., 1072., 1254., 1436., 1618., 1800.]])

## Bondes D

In [ ]:
m = len(plazos_bdm)
N_bd = (plazos_bdm // plazocupon_bdm) + 1  # número de cupones a pagar

# Inicialización de matrices/vectores
VTplazos_bdm = np.zeros(np.sum(N_bd))
contratos_bdmT = np.zeros(np.sum(N_bd))
plazocupon_bdmT = np.zeros(np.sum(N_bd))
tasafijaT_bd = np.zeros(np.sum(N_bd))
ulNomT_bd = np.zeros(np.sum(N_bd))
Xvp_bd = np.zeros((n, len(VTplazos_bdm)))
Xst_bd = np.zeros((n, len(VTplazos_bdm)))

# Cálculos adicionales
plazini_bd = plazos_bdm - plazocupon_bdm * (N_bd - 1)
ddv = plazocupon_bdm - plazini_bd
tfcupon = np.zeros(m)
tfcupondev = np.zeros(m)
tfcupgen = ((1 + tf_act / 360)**(plazocupon_bdm[0]) - 1) * 360 / plazocupon_bdm[0]

# Bucle para calcular tasas de cupón
for j in range(m):  # en Python los índices comienzan en 0
    tfcupondev[j] = ((np.prod(1 + tf_int[0:ddv[j]] / 360) - 1) * 360) / ddv[j]
    tfcupon[j] = (((1 + tfcupondev[j] * ddv[j] / 360) * (1 + tf_act / 360)**(plazocupon_bdm[0] - ddv[j]) - 1) * 360) / plazocupon_bdm[0]

print(tfcupondev)
print(tfcupon)

from scipy.interpolate import interp1d

for j in range(m):
    sum_N_bd = np.sum(N_bd[:j+1])
    sum_N_bd_prev = np.sum(N_bd[:j]) if j > 0 else 0

    if j == 0:
        VTplazos_bdm[:sum_N_bd] = np.arange(plazini_bd[j], plazos_bdm[j] + 1, plazocupon_bdm[j])
        contratos_bdmT[:sum_N_bd] = contratos_bdm[j]
        plazocupon_bdmT[:sum_N_bd] = plazocupon_bdm[j]
        ulNomT_bd[sum_N_bd - 1] = contratos_bdm[j]
        tasafijaT_bd[0] = tfcupon[j]
        tasafijaT_bd[1:sum_N_bd] = tfcupgen
    else:
        VTplazos_bdm[sum_N_bd_prev:sum_N_bd] = np.arange(plazini_bd[j], plazos_bdm[j] + 1, plazocupon_bdm[j])
        contratos_bdmT[sum_N_bd_prev:sum_N_bd] = contratos_bdm[j]
        plazocupon_bdmT[sum_N_bd_prev:sum_N_bd] = plazocupon_bdm[j]
        tasafijaT_bd[sum_N_bd_prev] = tfcupon[j]
        tasafijaT_bd[sum_N_bd_prev + 1:sum_N_bd] = tfcupgen
        ulNomT_bd[sum_N_bd - 1] = contratos_bdm[j]

for i in range(n):
    if itpl == 0:
        interpolator = interp1d(nodos, x_orig_gov.iloc[i, :], kind='linear', fill_value="interpolate")
        Xvp_bd[i, :] = interpolator(VTplazos_bdm)

        interpolator3 = interp1d(nodos3_bd, X3_orig_bd.iloc[i, :], kind='linear', fill_value="interpolate")
        Xst_bd[i, :] = interpolator3(VTplazos_bdm)
    else:
        # Código para interpolación 'talamb' (no definido en el ejemplo de R original)
        pass

X2_pr=pd.merge(X2_orig_bd, current_df, left_on='fecha', right_on="Date", how='inner').sort_values('Date', ascending=False)
X_bd_tc = (np.ones((n, len(contratos_bdmT))).T * (np.array(X2_pr[1]) / 100)).T
X_bd_ext = np.hstack((X_bd_tc, Xvp_bd, Xst_bd))


def bondeD(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, Xst, N, ulNomT):
    V0 = np.zeros(len(N))  # Inicializa V0, equivalente a matrix(0, 1, nrow(as.matrix(N))) en R
    V0f = (((contratosT * tasafijaT * (plazocuponT / 360)) + ulNomT) / (1 + (Xvp + Xst) * VTplazos / 360)) * nominal

    for j in range(len(N)):  # Cambia índices de 1:nrow(as.matrix(N)) a 0:(len(N)-1)
        if j == 0:  # Cambia de j==1 a j==0
            V0[j] = np.sum(V0f[:N[j]])
        else:
            V0[j] = np.sum(V0f[np.sum(N[:j]):np.sum(N[:j+1])])

    return V0

# Suponiendo que contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, Xst, y N ya están definidos
V0_bd = bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[0,], Xst_bd[0,], N_bd, ulNomT_bd)  # Cambia de Xvp[1, ] y Xst[1, ] a Xvp[0, :] y Xst[0, :]

print(V0_bd)

[0.11001729 0.11038289 0.11022157 0.10992531 0.11044562]
[0.10980679 0.11015967 0.10996162 0.10974557 0.11031451]
[ 22161.21115235 -29320.69510623  29377.07811909 -47181.61323033
  10480.40284521]
